In [1]:
import os
import glob
import subprocess as sb
import sys

In [2]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        

In [3]:
dockerhub_account='pinellolab'

In [4]:
dir_syn_dataset="./dataset/synthetic"
dir_real_dataset="./dataset/real"

dir_syn_results="./results/synthetic"
dir_real_results="./results/real"

ensure_dir(dir_syn_results)
ensure_dir(dir_real_results)

syn_datasets=[]
real_datasets=[]

metrics_syn=["correlation", "accuracy" ,"stability"]
metrics_real=["correlation", "accuracy" ,"stability"]

methods=["monocle2",]

FORCE_RERUN=False
DEBUG=True

In [5]:
#test methods on synthetic datasets
for dataset_dir in glob.glob("./dataset/synthetic/*"):
    if os.path.isdir(dataset_dir):
        syn_datasets.append(dataset_dir)

In [6]:
#test methods on real datasets
for dataset_dir in glob.glob("./dataset/real/*"):
    if os.path.isdir(dataset_dir):
        real_datasets.append(dataset_dir)

In [7]:
syn_datasets

['./dataset/synthetic/prosst_topology3_prosstt0']

In [8]:
real_datasets

[]

In [9]:
for dataset_dir in syn_datasets:
    dataset_name=os.path.basename(dataset_dir)
        
    print ('\n\n[Current Dataset]: %s\n' % dataset_name)
    method_input_filename=os.path.join(dataset_dir,'input.txt')
    metric_input_filename=os.path.join(dataset_dir,'truth.txt')
    
    
    if os.path.exists(method_input_filename) and \
        os.path.exists(metric_input_filename): 
    
        for method_name in methods:
            
            docker_image_method='%s/hca_method_%s' % (dockerhub_account, method_name)
            print ('  [Running Method]: %s (%s)\n' %
                (method_name, docker_image_method))

            
            output_dir= os.path.join(dir_syn_results,dataset_name,method_name)
            ensure_dir(output_dir)


            method_output_filename=os.path.join(output_dir ,'output.txt')
            
            
            #print (docker_image_method,dataset_path, dataset_name,output_folder)

            cmd_method_to_run='docker run \
            -v ${PWD}:/data -w /data %s run_method \
            %s %s'\
            % (docker_image_method,method_input_filename, method_output_filename )
           
            if DEBUG:
                print ( cmd_method_to_run)
            
            if os.path.exists(method_output_filename) and not FORCE_RERUN:
                print('    The method was already run on this dataset, skipping.\n')
                
            else:
                sb.check_output(cmd_method_to_run,shell=True)

                

            for metric_name in metrics_syn:
                
                docker_image_metric='%s/hca_score_%s' % (dockerhub_account, metric_name)
                print('    [Evaluating Metric]: %s (%s)\n' %( metric_name, docker_image_metric))

                if metric_name != 'stability':                
                    
                    metric_output_filename=os.path.join(output_dir,'hca_score_%s.txt' % metric_name)
                    cmd_metric_to_run='docker run \
                    -v  ${PWD}:/hca -w /hca %s -o %s -t %s  -s %s' \
                    %(docker_image_metric,method_output_filename, metric_input_filename, metric_output_filename)

                    if DEBUG:
                        print ('    '+cmd_metric_to_run+'\n')
                    
                    if os.path.exists(metric_output_filename) and not FORCE_RERUN:
                        print('      The metric was already run for this dataset method pair, skipping.\n')
                    
                    else:
                        sb.check_output(cmd_metric_to_run,shell=True)
                else:
                    pass
           
    else:
        print ('Input dataset is missing the required files: input.txt and or truth.txt')



[Current Dataset]: prosst_topology3_prosstt0

  [Running Method]: monocle2 (pinellolab/hca_method_monocle2)

docker run             -v ${PWD}:/data -w /data pinellolab/hca_method_monocle2 run_method             ./dataset/synthetic/prosst_topology3_prosstt0/input.txt ./results/synthetic/prosst_topology3_prosstt0/monocle2/output.txt
    [Evaluating Metric]: correlation (pinellolab/hca_score_correlation)

    docker run                     -v  ${PWD}:/hca -w /hca pinellolab/hca_score_correlation -o ./results/synthetic/prosst_topology3_prosstt0/monocle2/output.txt -t ./dataset/synthetic/prosst_topology3_prosstt0/truth.txt  -s ./results/synthetic/prosst_topology3_prosstt0/monocle2/hca_score_correlation.txt

    [Evaluating Metric]: accuracy (pinellolab/hca_score_accuracy)

    docker run                     -v  ${PWD}:/hca -w /hca pinellolab/hca_score_accuracy -o ./results/synthetic/prosst_topology3_prosstt0/monocle2/output.txt -t ./dataset/synthetic/prosst_topology3_prosstt0/truth.txt  -

In [ ]:
method_output_filename

In [ ]:
method_input_filename

In [ ]:
!ls ./results/synthetic/prosst_topology3_prosstt0/

In [ ]:
output_dir